In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
 %%bash
 module load texlive

In [3]:
import torch
from acctrack.tools.model_inference import TorchModelInference
from acctrack.tools.utils_graph import build_edges
from pathlib import Path

/global/homes/x/xju/.conda/envs/gnn4itk/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


uproot is not installed. AthenaRawRootReader will not be available.


## General Configurations

In [4]:
base_path = "/global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper"
model_path = base_path + "/TrainedModels/graph_construction-MetricLearning-v2.pt"
data_path = base_path + "/2023_uncorr_feature_store"

r_max, k_max = 0.1, 1000
knn_backend = "FAISS-CPU"
config_file = "metric_learning-nonoise.yaml"
embedding_file_name = "embedding.pt"

## Compare different KNN algorithms

### Create the embedding

In [4]:
model_infer = TorchModelInference(config_file, model_path, ".", name="ForTesting")

Training: Total 1 events in directory: /global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper/2023_uncorr_feature_store/trainset
Validation: Total 1 events in directory: /global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper/2023_uncorr_feature_store/valset
Test: Total 1 events in directory: /global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper/2023_uncorr_feature_store/testset


In [5]:
data = model_infer.data_reader_training.read(0)
node_features = model_infer.config["node_features"]
node_scales = torch.Tensor(model_infer.config["node_scales"])
input_data = torch.stack([data[x] for x in node_features], dim=-1).float()
features = input_data / node_scales
embedding = model_infer.model_reader.predict(features)

Reading file: /global/homes/x/xju/m3443/usr/xju/ITk/For2023Paper/2023_uncorr_feature_store/trainset/event000005367-graph.pyg


In [6]:
torch.save(embedding, embedding_file_name)

### Different algorithms

In [5]:

if Path(embedding_file_name).exists():
    embedding = torch.load(embedding_file_name)
    print(embedding.shape)
else:
    print("Run the embedding creation step first!")


torch.Size([349243, 32])


In [8]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-FlatL2")
edge_index.shape

torch.Size([2, 46795462])

In [9]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-QUANTIZED", nprobe=1)
edge_index.shape

torch.Size([2, 24308418])

In [10]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-QUANTIZED", nprobe=5)
edge_index.shape

torch.Size([2, 40693091])

In [11]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-QUANTIZED", nprobe=10)
edge_index.shape

torch.Size([2, 44498115])

In [12]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-QUANTIZED", nprobe=20)
edge_index.shape

torch.Size([2, 46364250])

In [11]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-QUANTIZED", nlist=1000, nprobe=10)
edge_index.shape

torch.Size([2, 39309573])

In [12]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=1000, nprobe=20)
edge_index.shape

torch.Size([2, 42728981])

In [13]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=500, nprobe=20)
edge_index.shape

torch.Size([2, 43659143])

In [14]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=10)
edge_index.shape

torch.Size([2, 46041135])

In [15]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=25)
edge_index.shape

torch.Size([2, 46785100])

In [16]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-IVF", nlist=50, nprobe=30)
edge_index.shape

torch.Size([2, 46791439])

In [7]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-CPU-scalarquantizer")
edge_index.shape

torch.Size([2, 40366750])

In [19]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FAISS-GPU")
edge_index.shape

torch.Size([2, 46795043])

In [16]:
edge_index = build_edges(embedding, r_max=r_max, k_max=k_max, backend="FRNN")
edge_index.shape

torch.Size([2, 46794019])